In [18]:
import torch
import cv2

In [19]:
model = torch.load('SegmentAnalysis.pth')
video = cv2.VideoCapture('../Videos/game_1.mp4')


In [20]:
import torch
import torch.nn as nn

# Define your model architecture
class EventSegmentationLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.classifier = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        logits = self.classifier(out)
        return logits

# --- Step 1: Set your model hyperparameters ---
input_dim = 1000     # replace with your actual input dimension
hidden_dim = 128     # replace with your actual hidden dimension
num_classes = 2     # replace with your actual number of classes

# --- Step 2: Initialize the model ---
model = EventSegmentationLSTM(input_dim, hidden_dim, num_classes)

# --- Step 3: Load the state_dict ---
state_dict_path = "SegmentAnalysis.pth"  # replace with your saved file path
state_dict = torch.load(state_dict_path, map_location='cpu')  # use map_location if no GPU
model.load_state_dict(state_dict)

# --- Step 4: Set model to eval mode if needed ---
model.eval()

print("Model loaded successfully!")


Model loaded successfully!


In [23]:
import cv2
import torch
import torchvision.transforms as T
from torchvision.models import resnet18

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load model
model = resnet18(pretrained=True).to(device)
model.eval()

# Transform (same as training)
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# Open video
cap = cv2.VideoCapture("../Videos/rallies_01.mp4")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Apply transforms
    img_tensor = transform(frame).unsqueeze(0).to(device)  # shape [1, 3, 224, 224]

    # Forward pass
    with torch.no_grad():
        output = model(img_tensor)

    # Convert output to class
    pred_idx = torch.argmax(output[0,0,:]).item()  # first batch, first timestep, all classes
    pred_class = "Class A" if pred_idx == 0 else "Class B"

    # Display prediction on frame
    cv2.putText(frame, f"Class: {pred_class}", (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


IndexError: too many indices for tensor of dimension 2